In [ ]:
import numpy as np
import sys,time
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy import signal
import progressbar
from time import sleep
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import math
import pandas as pd
import ase
from ase import io
from ase.build import bulk,diamond100
from ase.spacegroup import crystal
import numpy as np
from ase import Atoms,Atom
import os
import pickle
from ase.build.tools import sort

def crystal_structure(a,L):
    si = crystal(['Si'], basis=[(0,0,0)], spacegroup=227,
         cellpar=[a, a, a, 90., 90., 90.])
    return si * (L, L, L)

def create_lammps_file(crystal_structure,name):
    file = ase.io.write(name, crystal_structure, format="lammps-data")
    return file 

def read_velocities_file(namefile,frames, N, dimension):
    fileName = namefile
    velocities = np.zeros((frames, N, dimension))  
    fin = open(fileName, "r")
    for i in tqdm_notebook(range(frames),desc = 'read velocities file progress:'):
        sleep(0.0000000000000000000001)          
        for j in range(9):                          # lines should be excluded: 9 
            fin.readline()
        for k in range(N):
            line = fin.readline().split()[2:]
            line = [float(l) for l in line]
            velocities[i, k] = line    
    return velocities  

def read_file(namefile):
    file = open(namefile, 'r') 
    lines = file.readlines()
    x = []
    y = [] 
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    file.close() 
    return x,y




def read_rdf_adf_file(namefile, bins, steps, n):
    array = np.zeros((steps, bins, n))
    fileName = namefile  
    fin = open(fileName, "r")
    for i in tqdm_notebook(range(steps),desc = 'read file progress:'):
        sleep(0.0000000000000000000001)   
        if (i==0):    
            for j in range(4):                      # lines should be excluded: 4 
                fin.readline()
            for k in range(bins):
                line = fin.readline().split()[0:]
                line = [float(l) for l in line] 
                array[i, k] = line
        else:
            for j in range(1):                      # lines should be excluded: 1 
                fin.readline()
            for k in range(bins):
                line = fin.readline().split()[0:]
                line = [float(l) for l in line]
                array[i, k] = line
    return array                

def calculate_VACF(velocities, frames, N):
    VACF = np.zeros(frames,dtype=float)
    for atom in tqdm_notebook(range(N),desc = 'calculate VACF progress:'):
        sleep(0.0000000000000000000001)
        for time in range (frames):
            VACF[time] += (velocities[0,atom,:] @ velocities[time,atom,:])/(velocities[0,atom,:] @ velocities[0,atom,:])
                
    VACF /= VACF[0]
    return VACF

def calculate_DOS(VACF, dt, points, std):
    n = VACF.size
    fourier = np.fft.rfft(VACF)
    fourier = fourier/max(abs(fourier))
    fft_v = np.conj(fourier) * fourier
    fft_v =np.abs(fft_v)
    freq = np.fft.rfftfreq(n, dt)/1.0E12

    win = signal.windows.gaussian(points, std)
    DOS = signal.convolve(fft_v, win, mode='same') / sum(win)
    
    return freq,DOS

def create_graphs(namegraph, t, VACF, Freq, DOS, RDF, ADF, binRDF, binADF, N):
    fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(namegraph)
    ax1.plot(t, VACF)
    ax1.set_title("VACF")
    ax1.set(xlabel='Time [ps]', ylabel= r'${<vi(0)\cdot vi(t)>}$')
    
    ax2.plot(RDF[binRDF,:,1],RDF[binRDF,:,2])
    ax2.set_title("RDF")
    ax2.set(xlabel='r(Å)', ylabel= 'RDF')
    
    ax3.plot(Freq,DOS)
    ax3.set_title("Total DOS")
    ax3.set(xlabel='Frequency THz', ylabel= 'Density Of States A.U.')
    ax3.set_xlim((0, 20))
    ax3.set_ylim(bottom=0)
    
    ax4.plot(ADF[binADF,:,1],ADF[binADF,:,2])
    ax4.set_title("ADF")
    ax4.set(xlabel='θ°', ylabel= 'ADF')
    
    flag = '%s' % str(namegraph) 
    plt.savefig("%s.jpg" % flag, dpi=100)


def create_graphs_SiO2(namegraph, t, VACF, Freq, DOS, RDF, ADF1, ADF2, binRDF, binADF, N):
    fig = plt.figure(figsize=(18, 12))
    fig.suptitle(namegraph)
                
    plt.subplot(231)             
    plt.plot(t, VACF)
    plt.title("VACF")
    plt.xlabel('Time [ps]')
    plt.ylabel(r'${<vi(0)\cdot vi(t)>}$')
    
                 
    plt.subplot(2,3,2)            
    plt.plot(RDF[binRDF,:,1],RDF[binRDF,:,2])
    plt.title("RDF")
    plt.xlabel('r(Å)')
    plt.ylabel('RDF')
    
    plt.subplot(2,3,3)             
    plt.plot(Freq,DOS)
    plt.title("Total DOS")
    plt.xlabel('Frequency THz')
    plt.ylabel('Density Of States A.U.')
    plt.xlim((0, 50))
    plt.ylim(bottom=0)

    plt.subplot(2,3,4)
    plt.plot(ADF1[binADF,:,1],ADF1[binADF,:,2])
    plt.title("ADF O-Si-O")
    plt.xlabel('θ°')
    plt.ylabel('ADF')
    
    plt.subplot(2,3,5)
    plt.plot(ADF2[binADF,:,1],ADF2[binADF,:,2])
    plt.title("ADF Si-O-Si")
    plt.xlabel('θ°')
    plt.ylabel('ADF')             
                 
    flag = '%s' % str(namegraph) 
    plt.savefig("%s.jpg" % flag, dpi=100)


#create lammps data for input files
#lattice constant for Si is a=5.47
#3 structures for different atoms and 1 surface in this case
#nf = the number of files with lammps data 

a = float(input("Enter lattice constant: "))
nf = 3                                        
for i in (range(nf)):
    L = int(input("Enter length for supercell: "))
    name = str(input("Enter name for lammps data file: ")) 
    Si = crystal_structure(a, L)
    create_lammps_file(Si,name)


Si_SLAB = diamond100('Si', size=(25,25,4),a=a,vacuum=1, orthogonal=True, periodic=True)
create_lammps_file(Si_SLAB,name="Si.lammpsSLAB")

#After create files, run lammps simulations

# For SiO2 from DFT OUTCAR data 

atoms = io.read('OUTCAR_comp')
atoms *= (1, 1, 1)
bec = sort(atoms) 
ase.io.lammpsdata.write_lammps_data('SiO2.lammps', bec)

slab = surface(bec, (2,2,1), 20,  vacuum=1)
ase.io.cif.write_cif('cif_slab.cif', slab)
ase.io.lammpsdata.write_lammps_data('SiO2.lammpsSLAB', slab)

runs = 37500                      # for 30 ps with dt=0.0008
snapshoots = 4                    # take snapshoot every 4 runs
Frames = int(runs/snapshoots)     # sum of snapshoots
dt = 4*8*10**-16                  # convert to ps
t = np.arange(Frames)*dt*1.0E12   # time array in ps
Points = 20                       # numper of points for fourier transform
Std = 6000                        # std for 30 ps simulation 
binsRDF = 100                     # bins at every step for RDF
n1 = 3                            # length of columns velocities 
n2 = 4                            # number of columns RDF, ADF
steps = 375                       # steps for RDF with lammps format ave/time 10 10 100 
binsADF = 45                      # bins at every step for ADF


# for Si

files = int(input("Enter number of files simulations "))

for i in (range(files)):
    N = int(input("Enter number of atoms "))
    trjfile = str(input("Enter the name of trajectory file "))
    RDFfile = str(input("Enter the name of RDF file "))
    ADFfile = str(input("Enter the name of ADF file "))
    namegraph = str(input("Enter the name of graph "))

    velocities = read_velocities_file(trjfile, Frames, N, n1)  

    VACF = calculate_VACF(velocities,Frames,N)

    Freq,DOS = calculate_DOS(VACF, dt,Points, Std)
    
    namefile = str(input("Enter the name file to save DOS,Freq "))
    file = open(namefile, "w")
    for index in range(len(Freq)):
        file.write(str(Freq[index]) + " " + str(DOS[index]) + "\n")
    file.close()

    
    #for RDF 
    RDF = read_rdf_adf_file(RDFfile, binsRDF, steps, n2)
    #for ADF 
    ADF = read_rdf_adf_file(ADFfile, binsADF, steps, n2)
    #graphs
    create_graphs(namegraph, t, VACF, Freq, DOS, RDF, ADF, binsRDF, binsADF, N)

#For SiO2

files = int(input("Enter number of files simulations "))

for i in (range(files)):
    N = int(input("Enter number of atoms "))
    trjfile = str(input("Enter the name of trajectory file "))
    RDFfile = str(input("Enter the name of RDF file "))
    ADF1file = str(input("Enter the name file of  O-Si-O "))
    ADF2file = str(input("Enter the name file of  Si-O-Si  "))
    namegraph = str(input("Enter the name of graph "))

    velocities = read_velocities_file(trjfile, Frames, N, n1)  

    VACF = calculate_VACF(velocities,Frames,N)

    Freq,DOS = calculate_DOS(VACF, dt,Points, Std)
    
    namefile = str(input("Enter the name file to save DOS,Freq "))
    file = open(namefile, "w")
    for index in range(len(Freq)):
        file.write(str(Freq[index]) + " " + str(DOS[index]) + "\n")
    file.close()

    
    #for RDF 
    RDF = read_rdf_adf_file(RDFfile, binsRDF, steps, n2)
    #for ADF 
    ADF1 = read_rdf_adf_file(ADF1file, binsADF, steps, n2)
    
    ADF2 = read_rdf_adf_file(ADF2file, binsADF, steps, n2)
    #graphs
    create_graphs(namegraph, t, VACF, Freq, DOS, RDF, ADF1, ADF2, binsRDF, binsADF, N)


# when namefile1, namefile2,... , give  name of the files for Si or SiO2 
namefile1 = "namefile1"
namefile2 = "namefile2"


Freq1,DOS1 = read_file(namefile1)
Freq2,DOS2 = read_file(namefile2)

fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(Freq1,DOS1,'--', label = 'namegraph1',linewidth=1.2)
ax.plot(Freq2,DOS2,':',label = 'namegraph2',linewidth= 1)
ax.set_title("DOS")
ax.set_xlabel('Frequency THz')
ax.set_ylabel('Density Of States A.U.')
ax.set_xlim((0, 20))
ax.set_ylim(bottom=0)
ax.legend();    

plt.savefig("Total.jpg", dpi=100)
